In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load your base notebook (embeddings, df, helpers)
LIGHT_IMPORT = True
%run -i "Calc_Embeddings_Model Training.ipynb"

# quick sanity
print("df shape:", df.shape if 'df' in globals() else 'df missing')
print("have get_matches?", 'get_matches' in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

In [5]:
# --- knobs (adjust later if you want) ---
CUPID_TOP_K   = 20
CUPID_POOL_K  = 500
CUPID_MAX_KM  = 50

def cupid_candidate_pool(user_id: int,pool_k: int = CUPID_POOL_K,max_km: float = CUPID_MAX_KM) -> pd.DataFrame:
    """
    Build the candidate pool using only hard gates:
      - orientation/sex as the user specifies (inside get_matches)
      - age range
      - location radius
    NOTE: No lifestyle gating here. We'll handle lifestyle softly during scoring.
    """
    cands = get_matches(user_id=user_id, k=pool_k,use_orientation=True,use_location=True, max_km=max_km,use_age=True,
        # all lifestyle OFF here -> soft handling later
        use_diet=False, use_drinks=False, use_smokes=False, use_drugs=False,
        drinks_strict=False, smokes_strict=False, drugs_strict=False,
        allow_missing=True).copy()

    # ensure a minimal set of columns is present (keep others if they exist)
    base_cols = ["user_id", "age", "sex", "orientation", "location"]
    for col in base_cols:
        if col not in cands.columns:
            cands[col] = np.nan

    # return with base cols first, then any extra columns that came along
    ordered = base_cols + [c for c in cands.columns if c not in base_cols]
    return cands[ordered]

In [ ]:
"""
sets a few simple knobs (CUPID_TOP_K, CUPID_POOL_K, CUPID_MAX_KM).
defines cupid_candidate_pool(...), which calls your get_matches with only hard gates (orientation, age, location). 
lifestyle is intentionally left out here, it will be soft later in the scoring step.
runs a quick smoke test so you can see the pool size and first 10 rows
"""

In [6]:
u = 14
pool = cupid_candidate_pool(user_id=u)
print("pool size:", len(pool))
pool.head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 2478
Unfiltered: 2478 | After location filter (≤50 km): 2457
After location (≤50 km): 2457
After age rule: 2225
pool size: 500


,user_id,age,sex,orientation,location,text_sim,diet_c,drinks_c,smokes_c,drugs_c,lat,lon
2,38159,32,m,straight,"san carlos, california",0.742732,,light,none,none,37.504936,-122.261823
7,40022,25,m,straight,"san francisco, california",0.736307,omnivore,heavy,light,,37.779259,-122.419329
9,25427,29,m,straight,"mountain view, california",0.735156,vegetarian,light,none,none,37.389389,-122.083210
10,3717,37,m,straight,"palo alto, california",0.734164,,,light,,37.444329,-122.159847
11,28563,29,m,straight,"san francisco, california",0.732910,,light,none,none,37.779259,-122.419329
12,39450,25,m,straight,"san francisco, california",0.732575,omnivore,,none,none,37.779259,-122.419329
16,23663,27,m,straight,"san mateo, california",0.730931,omnivore,light,none,none,37.496904,-122.333057
20,15046,36,m,straight,"san francisco, california",0.728997,omnivore,light,none,none,37.779259,-122.419329
21,33656,26,m,straight,"san francisco, california",0.728639,omnivore,light,none,none,37.779259,-122.419329
26,46695,31,m,straight,"palo alto, california",0.726020,vegetarian,light,none,none,37.444329,-122.159847


In [7]:
#Base compatibility (text/age/lifestyle/location)
# Uses your compute_component_scores to get component columns, then blends them into a single base_score (no freshness/explore yet).

# weights for the base
CUPID_W = {"text": 0.60, "age": 0.10, "life": 0.15, "loc": 0.15}

def _ensure_text_sim(cands: pd.DataFrame) -> pd.DataFrame:
    """
    make sure text_sim is numeric and clipped to [0,1].
    """
    c = cands.copy()
    c["text_sim"] = pd.to_numeric(c.get("text_sim", 0.0), errors="coerce").fillna(0.0).clip(0, 1)
    return c

def cupid_base_compat(user_id: int,pool_k: int = CUPID_POOL_K,max_km: float = CUPID_MAX_KM,
                      weights: dict = CUPID_W) -> pd.DataFrame:
    """
    1) Build hard-gated pool (orientation/age/location)
    2) Compute component scores (text_sim, age_score, lifestyle_score, loc_score)
    3) Blend into base_score = 0.60*text + 0.10*age + 0.15*life + 0.15*loc
    Returns a dataframe sorted by base_score (desc).
    """
    # (1) pool
    pool = cupid_candidate_pool(user_id=user_id, pool_k=pool_k, max_km=max_km)

    # (2) component scores (lifestyle kept soft here)
    c = compute_component_scores(user_id=user_id, cands=pool, max_km=max_km,
        drinks_strict=False, smokes_strict=False, drugs_strict=False, allow_missing=True)
    c = _ensure_text_sim(c)

    # make sure required columns exist
    for col in ["age_score", "lifestyle_score", "loc_score"]:
        if col not in c.columns:
            c[col] = 0.0

    # (3) weighted blend
    w = weights
    c["base_score"] = (w["text"] * c["text_sim"] +w["age"]  * c["age_score"] +w["life"] * c["lifestyle_score"] +
        w["loc"]  * c["loc_score"])

    need = ["user_id", "base_score", "text_sim", "age_score", "lifestyle_score", "loc_score",
            "age", "sex", "orientation", "location"]
    for col in need:
        if col not in c.columns:
            c[col] = np.nan

    return c[need].sort_values("base_score", ascending=False).reset_index(drop=True)

In [8]:
u = 14
base = cupid_base_compat(user_id=u)
print("rows:", len(base))
base.head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 2478
Unfiltered: 2478 | After location filter (≤50 km): 2457
After location (≤50 km): 2457
After age rule: 2225
rows: 500


,user_id,base_score,text_sim,age_score,lifestyle_score,loc_score,age,sex,orientation,location
0,38159,0.845639,0.742732,1.0,1.0,1.0,32,m,straight,"san carlos, california"
1,40022,0.841784,0.736307,1.0,1.0,1.0,25,m,straight,"san francisco, california"
2,3717,0.840498,0.734164,1.0,1.0,1.0,37,m,straight,"palo alto, california"
3,28563,0.839746,0.732910,1.0,1.0,1.0,29,m,straight,"san francisco, california"
4,39450,0.839545,0.732575,1.0,1.0,1.0,25,m,straight,"san francisco, california"
5,23663,0.838559,0.730931,1.0,1.0,1.0,27,m,straight,"san mateo, california"
6,15046,0.837398,0.728997,1.0,1.0,1.0,36,m,straight,"san francisco, california"
7,33656,0.837184,0.728639,1.0,1.0,1.0,26,m,straight,"san francisco, california"
8,16398,0.835587,0.725979,1.0,1.0,1.0,35,m,straight,"palo alto, california"
9,35672,0.835263,0.725438,1.0,1.0,1.0,25,m,straight,"oakland, california"


In [11]:
# Freshness 
# - Format last_online to YYYY-MM-DD
# - Build recency_norm ∈ [0,1] on the master df
# - Add freshness to base_score → cupid_score_fresh

CUPID_G_FRESH = 0.10  # weight for freshness

# 1) Clean + parse dates on master df
df["last_online_clean"] = df["last_online"].astype(str).str.slice(0, 10)  # YYYY-MM-DD
df["last_online_dt"] = pd.to_datetime(df["last_online_clean"], errors="coerce", utc=True)

# 2) Normalize to [0,1] over the dataset
min_dt = df["last_online_dt"].min()
max_dt = df["last_online_dt"].max()

if pd.notna(min_dt) and pd.notna(max_dt) and max_dt > min_dt:
    span_sec = (max_dt - min_dt).total_seconds()
    df["recency_norm"] = ((df["last_online_dt"] - min_dt).dt.total_seconds() / span_sec).fillna(0.0)
else:
    # if all dates are the same or missing, set to 0.0
    df["recency_norm"] = 0.0

# 3) Merge into base and add freshness bump
fresh = base.merge(df[["user_id", "recency_norm"]], on="user_id", how="left")
fresh["recency_norm"] = fresh["recency_norm"].fillna(0.0).clip(0, 1)
fresh["cupid_score_fresh"] = fresh["base_score"] + CUPID_G_FRESH * fresh["recency_norm"]

print("Built recency_norm. Preview:")
fresh[["user_id","base_score","recency_norm","cupid_score_fresh"]].head(10)

Built recency_norm. Preview:


,user_id,base_score,recency_norm,cupid_score_fresh
0,38159,0.845639,0.997297,0.945369
1,40022,0.841784,0.305405,0.872325
2,3717,0.840498,0.994595,0.939958
3,28563,0.839746,0.348649,0.874611
4,39450,0.839545,0.578378,0.897383
5,23663,0.838559,0.567568,0.895315
6,15046,0.837398,0.997297,0.937128
7,33656,0.837184,0.537838,0.890967
8,16398,0.835587,0.997297,0.935317
9,35672,0.835263,0.997297,0.934992


In [12]:
#  Synthesize encounters & popularity, then compute cupid_base
# - Reproducible synthetic schedule_strength and right_swipe_rate
# - Adds UCB bonus and popularity penalty, producing `cupid_base`

CUPID_ALPHA_UCB = 0.15   # exploration strength
CUPID_BETA_POP  = 0.20   # popularity penalty
CUPID_SYN_SEED  = 7      # reproducible

rng = np.random.default_rng(CUPID_SYN_SEED)

def cupid_add_explore_pop_synth(fresh_df: pd.DataFrame) -> pd.DataFrame:
    """
    Build synthetic:
      - schedule_strength: higher if more recent activity, with noise (0..~20)
      - right_swipe_rate: function of text_sim + lifestyle_score, with tiny noise (0..1)
    Then compute:
      - ucb_bonus = α / sqrt(1 + schedule_strength)
      - pop_penalty = β * right_swipe_rate
      - cupid_base  = cupid_score_fresh + ucb_bonus - pop_penalty
    """
    out = fresh_df.copy()

    # Make sure needed columns exist
    for col in ["recency_norm", "text_sim", "lifestyle_score", "cupid_score_fresh"]:
        if col not in out.columns:
            out[col] = 0.0

    # Synthetic encounters: more recent users tend to have more encounters
    # scale to ~0..20 with some randomness
    base_enc = 14 * out["recency_norm"] + 2 * rng.random(len(out))
    out["schedule_strength"] = np.floor(np.clip(base_enc, 0, None)).astype(int)

    # Synthetic popularity: driven by text & lifestyle; small noise
    rrate = (0.10 + 0.65 * out["text_sim"] + 0.15 * out["lifestyle_score"] + 0.03 * rng.standard_normal(len(out)))
    out["right_swipe_rate"] = np.clip(rrate, 0.0, 1.0)

    # Exploration bonus (bigger for low-encounter users)
    out["ucb_bonus"]   = CUPID_ALPHA_UCB / np.sqrt(1.0 + out["schedule_strength"])
    # Popularity penalty (downweight very over-exposed profiles)
    out["pop_penalty"] = CUPID_BETA_POP * out["right_swipe_rate"]

    # Final per-candidate pre-slate score
    out["cupid_base"]  = out["cupid_score_fresh"] + out["ucb_bonus"] - out["pop_penalty"]

    # peek
    print("Synthetic explore+pop added. Preview:")
    display(out[["user_id","recency_norm","text_sim","lifestyle_score","schedule_strength","right_swipe_rate",
                 "ucb_bonus","pop_penalty","cupid_base"]].head(8))
    return out

# %% [markdown]
# CUPID — Cell 3b (alt): Synthesize encounters & popularity, then compute cupid_base
# - No Elo code required
# - Reproducible synthetic schedule_strength and right_swipe_rate
# - Adds UCB bonus and popularity penalty, producing `cupid_base`

# %%
import numpy as np
import pandas as pd

CUPID_ALPHA_UCB = 0.15   # exploration strength
CUPID_BETA_POP  = 0.20   # popularity penalty
CUPID_SYN_SEED  = 7      # reproducible

rng = np.random.default_rng(CUPID_SYN_SEED)

def cupid_add_explore_pop_synth(fresh_df: pd.DataFrame) -> pd.DataFrame:
    """
    Build synthetic:
      - schedule_strength: higher if more recent activity, with noise (0..~20)
      - right_swipe_rate: function of text_sim + lifestyle_score, with tiny noise (0..1)
    Then compute:
      - ucb_bonus = α / sqrt(1 + schedule_strength)
      - pop_penalty = β * right_swipe_rate
      - cupid_base  = cupid_score_fresh + ucb_bonus - pop_penalty
    """
    out = fresh_df.copy()

    # Make sure needed columns exist
    for col in ["recency_norm", "text_sim", "lifestyle_score", "cupid_score_fresh"]:
        if col not in out.columns:
            out[col] = 0.0

    # Synthetic encounters: more recent users tend to have more encounters
    # scale to ~0..20 with some randomness
    base_enc = 14 * out["recency_norm"] + 2 * rng.random(len(out))
    out["schedule_strength"] = np.floor(np.clip(base_enc, 0, None)).astype(int)

    # Synthetic popularity: driven by text & lifestyle; small noise
    rrate = (
        0.10 + 0.65 * out["text_sim"] + 0.15 * out["lifestyle_score"] + 0.03 * rng.standard_normal(len(out))
    )
    out["right_swipe_rate"] = np.clip(rrate, 0.0, 1.0)

    # Exploration bonus (bigger for low-encounter users)
    out["ucb_bonus"]   = CUPID_ALPHA_UCB / np.sqrt(1.0 + out["schedule_strength"])
    # Popularity penalty (downweight very over-exposed profiles)
    out["pop_penalty"] = CUPID_BETA_POP * out["right_swipe_rate"]

    # Final per-candidate pre-slate score
    out["cupid_base"]  = out["cupid_score_fresh"] + out["ucb_bonus"] - out["pop_penalty"]

    # peek
    print("Synthetic explore+pop added. Preview:")
    display(out[["user_id","recency_norm","text_sim","lifestyle_score",
                 "schedule_strength","right_swipe_rate",
                 "ucb_bonus","pop_penalty","cupid_base"]].head(8))
    return out

# --- test run it on current `fresh` ---
scored = cupid_add_explore_pop_synth(fresh)

Synthetic explore+pop added. Preview:


,user_id,recency_norm,text_sim,lifestyle_score,schedule_strength,right_swipe_rate,ucb_bonus,pop_penalty,cupid_base
0,38159,0.997297,0.742732,1.0,15,0.755185,0.037500,0.151037,0.831832
1,40022,0.305405,0.736307,1.0,6,0.753211,0.056695,0.150642,0.778377
2,3717,0.994595,0.734164,1.0,15,0.698367,0.037500,0.139673,0.837784
3,28563,0.348649,0.732910,1.0,5,0.684678,0.061237,0.136936,0.798912
4,39450,0.578378,0.732575,1.0,8,0.715530,0.050000,0.143106,0.804277
5,23663,0.567568,0.730931,1.0,9,0.766839,0.047434,0.153368,0.789382
6,15046,0.997297,0.728997,1.0,13,0.639261,0.040089,0.127852,0.849365
7,33656,0.537838,0.728639,1.0,9,0.739414,0.047434,0.147883,0.790519


In [16]:
#  Reciprocal keyword boost (shared interests) — log-scale bonus
# - Extract keyword sets from bio_text (user & candidates)
# - Add diminishing-returns bonus via log1p(shared)
# - Output columns: kw_shared, kw_bonus, cupid_score_kw, cupid_pre_slate

import re

# Small, interpretable keyword lexicon (extend anytime)
CUPID_KEYWORDS = {"hiking","hike","run","running","walk","walking","gym","fitness","yoga","climb","biking",
                  "cycling","swim","travel","trip","explore","roadtrip","camp","music","concert","festival",
                  "dj","guitar","piano","sing","movie","movies","film","netflix","series","tv","anime",
                  "art","museum","gallery","paint","painting","draw","photography","food","cook","cooking",
                  "bake","coffee","brunch","restaurant","reading","books","book","boardgames","games","gaming",
                  "dog","dogs","cat","cats","pets"}
 
CUPID_DELTA_KW = 0.03  # base factor for keyword bonus (tunable)

def _kw_set(text: str) -> set:
    """Lowercase, tokenize, keep only words in CUPID_KEYWORDS."""
    if not isinstance(text, str) or not text.strip():
        return set()
    tokens = re.findall(r"[a-zA-Z]+", text.lower())
    return {t for t in tokens if t in CUPID_KEYWORDS}

def cupid_add_keywords_log(base_df: pd.DataFrame, user_id: int) -> pd.DataFrame:
    """
    Add keyword overlap bonus:
      kw_bonus = δ * log(1 + #shared_keywords)
    Produces: kw_shared, kw_bonus, cupid_score_kw, cupid_pre_slate
    """
    out = base_df.copy()

    # Build a fast lookup for bios
    bio_map = df.set_index("user_id")["bio_text"].to_dict()

    # User keyword set
    user_bio = bio_map.get(int(user_id), "")
    user_kw  = _kw_set(user_bio)

    # Overlap count per candidate
    def _overlap(uid):
        cand_bio = bio_map.get(int(uid), "")
        return len(user_kw & _kw_set(cand_bio))

    out["kw_shared"] = out["user_id"].astype(int).map(_overlap)

    # Log-scale bonus: grows with shared interests but with diminishing returns
    out["kw_bonus"] = CUPID_DELTA_KW * np.log1p(out["kw_shared"])

    # Add into score
    if "cupid_base" not in out.columns:
        raise KeyError("cupid_add_keywords_log expects 'cupid_base' in the input dataframe.")
    out["cupid_score_kw"] = out["cupid_base"] + out["kw_bonus"]

    # Alias for next step (pre-slate score before fairness/diversity)
    out["cupid_pre_slate"] = out["cupid_score_kw"]

    print("Keyword boost (log-scale) added. Preview:")
    display(out[["user_id","kw_shared","kw_bonus","cupid_base","cupid_score_kw"]].head(10))
    return out

scored_kw = cupid_add_keywords_log(scored, user_id=u)

Keyword boost (log-scale) added. Preview:


,user_id,kw_shared,kw_bonus,cupid_base,cupid_score_kw
0,38159,1,0.020794,0.831832,0.852626
1,40022,5,0.053753,0.778377,0.832130
2,3717,2,0.032958,0.837784,0.870743
3,28563,2,0.032958,0.798912,0.831871
4,39450,3,0.041589,0.804277,0.845866
5,23663,4,0.048283,0.789382,0.837665
6,15046,3,0.041589,0.849365,0.890954
7,33656,4,0.048283,0.790519,0.838802
8,16398,4,0.048283,0.834894,0.883178
9,35672,5,0.053753,0.823173,0.876926


In [17]:
# In‑slate diversity (greedy with similarity penalty)
# Input:  scored_kw  (must contain 'cupid_pre_slate'; will fetch bios from df)
# Method:
#   - Start from the highest 'cupid_pre_slate'
#   - For each next pick, compute Jaccard similarity between each remaining
#     candidate's keyword set and the sets already in the slate.
#   - Apply penalty = λ * max_similarity_to_slate
#   - Select argmax( score - penalty )
# Output: 'cupid_final' with diagnostic columns.

DIVERSITY_LAMBDA = 0.30  # λ — strength of the penalty (0.2–0.4 works well)

def _kw_set(text: str, lexicon) -> set:
    if not isinstance(text, str) or not text.strip():
        return set()
    toks = re.findall(r"[a-zA-Z]+", text.lower())
    return {t for t in toks if t in lexicon}

def _jaccard(a: set, b: set) -> float:
    if not a and not b:
        return 0.0
    inter = len(a & b)
    union = len(a | b)
    return inter / union if union else 0.0

def cupid_build_diverse_slate(pre_df: pd.DataFrame,k: int = CUPID_TOP_K,lexicon=CUPID_KEYWORDS,
                              lam: float = DIVERSITY_LAMBDA) -> pd.DataFrame:
    """
    Greedy diverse selection on 'cupid_pre_slate'.
    Returns a DataFrame with columns:
      ['user_id','cupid_pre_slate','div_penalty','cupid_final','kw_set','kw_shared']
    """
    C = pre_df.copy()
    if "cupid_pre_slate" not in C.columns:
        raise KeyError("Expected 'cupid_pre_slate' in input dataframe (run keyword step first).")

    # Bring bios for keyword sets
    if "bio_text" not in C.columns:
        C = C.merge(df[["user_id","bio_text"]], on="user_id", how="left")

    # Precompute keyword sets for each candidate
    bio_map = C.set_index("user_id")["bio_text"].to_dict()
    kw_map  = {int(uid): _kw_set(bio_map.get(int(uid), ""), lexicon) for uid in C["user_id"].astype(int)}

    # Work arrays
    C = C.sort_values("cupid_pre_slate", ascending=False).reset_index(drop=True)
    taken_ids = []
    taken_kw  = []

    # For diagnostics
    penalties = {}
    final_scores = {}

    for _ in range(min(k, len(C))):
        best_idx, best_val, best_pen = None, -1e9, 0.0

        for idx, row in C.iterrows():
            uid = int(row["user_id"])
            if uid in taken_ids:
                continue

            # similarity penalty: compare to already chosen sets
            K = kw_map.get(uid, set())
            if taken_kw:
                sim = max((_jaccard(K, T) for T in taken_kw), default=0.0)
            else:
                sim = 0.0
            pen = lam * sim
            val = float(row["cupid_pre_slate"]) - pen

            if val > best_val:
                best_val, best_idx, best_pen = val, idx, pen

        # take the best one
        if best_idx is None:
            break
        chosen = C.loc[best_idx].copy()
        uid = int(chosen["user_id"])
        taken_ids.append(uid)
        taken_kw.append(kw_map.get(uid, set()))
        penalties[uid] = best_pen
        final_scores[uid] = best_val

    # Build slate frame
    slate = C[C["user_id"].isin(taken_ids)].copy()
    slate["div_penalty"] = slate["user_id"].map(penalties).fillna(0.0)
    slate["cupid_final"] = slate["user_id"].map(final_scores)
    # handy columns for display
    slate["kw_set"] = slate["user_id"].map(kw_map)
    slate = slate.sort_values("cupid_final", ascending=False).reset_index(drop=True)

    cols = ["user_id","cupid_pre_slate","div_penalty","cupid_final","kw_set","sex","age","location"]
    for c in cols:
        if c not in slate.columns:
            slate[c] = np.nan
    return slate[cols]

# --- build a diverse top-k from your current pre-slate ---
diverse_slate = cupid_build_diverse_slate(scored_kw, k=20)
display(diverse_slate.head(10))


,user_id,cupid_pre_slate,div_penalty,cupid_final,kw_set,sex,age,location
0,15046,0.890954,0.000000,0.890954,"{music, concert, food, coffee, book, tv}",m,36,"san francisco, california"
1,18838,0.854691,0.000000,0.854691,"{books, movies, games, reading}",m,29,"san francisco, california"
2,3717,0.870743,0.037500,0.833243,"{travel, music, books, pets, walking}",m,37,"palo alto, california"
3,33504,0.818558,0.000000,0.818558,"{explore, dogs, art, walk, film}",m,32,"san francisco, california"
4,8486,0.877945,0.064286,0.813659,"{music, concert, books, cook, movies, art, ser...",m,24,"walnut creek, california"
5,38159,0.852626,0.042857,0.809769,"{trip, travel, cooking, reading}",m,32,"san carlos, california"
6,31015,0.882573,0.075000,0.807573,"{music, books, cooking, gym, food}",m,31,"san francisco, california"
7,16307,0.875107,0.069231,0.805876,"{travel, music, books, cooking, dj, hiking, ru...",m,32,"san francisco, california"
8,30473,0.805774,0.000000,0.805774,{},m,30,"san francisco, california"
9,24116,0.805322,0.000000,0.805322,"{yoga, biking}",m,30,"alameda, california"


In [28]:
#Pretty final table with shared interests & notes
# Inputs: diverse_slate (from Cell 5b), df (master), u (target user_id)
# Output: final_table (nice display)

DISPLAY_SHARED_MAX = 4

def user_kw_set(user_id: int) -> set:
    bio = df.loc[df["user_id"] == user_id, "bio_text"]
    bio = str(bio.iloc[0]) if len(bio) else ""
    toks = re.findall(r"[a-zA-Z]+", bio.lower())
    return {t for t in toks if t in CUPID_KEYWORDS}

def shared_kw_str(user_kw: set, cand_kw: set, limit: int = DISPLAY_SHARED_MAX) -> str:
    shared = sorted(user_kw & (cand_kw or set()))
    if not shared:
        return ""
    return ", ".join(shared[:limit]) + (" ..." if len(shared) > limit else "")

def cupid_finalize_table_simple(diverse_df: pd.DataFrame, user_id: int, k: int = CUPID_TOP_K, serendipity_n: int = 2) -> pd.DataFrame:
    C = diverse_df.copy()
    C["user_id"] = C["user_id"].astype(int)

    # main picks
    main_n = max(0, k - serendipity_n)
    main_df = C.sort_values("cupid_final", ascending=False).head(main_n).copy()
    main_df["slot"] = "matched"

    # serendipity from remaining (already hard‑filtered earlier)
    remaining = C[~C["user_id"].isin(main_df["user_id"])].copy()
    u_kw = user_kw_set(user_id)
    remaining["shared_cnt"] = remaining["kw_set"].apply(lambda s: len(u_kw & s))
    ser_pool = remaining[(remaining.get("recency_norm", 0) > 0.6) & (remaining["shared_cnt"] >= 1)]
    if len(ser_pool) < serendipity_n:
        ser_pool = pd.concat([ser_pool, remaining]).drop_duplicates("user_id")
    ser_df = ser_pool.sample(n=min(serendipity_n, len(ser_pool)), random_state=42).copy()
    ser_df["slot"] = "suggested"

    # combine
    final = pd.concat([main_df, ser_df], ignore_index=True).drop_duplicates("user_id")

    # drop conflicting cols before merge so we keep master values
    for col in ["age","sex","location","job","religion","bio_text"]:
        if col in final.columns:
            final = final.drop(columns=[col])

    # enrich from master df
    enrich = df[["user_id","age","sex","location","job","religion","bio_text"]].copy()
    enrich["user_id"] = enrich["user_id"].astype(int)
    final = final.merge(enrich, on="user_id", how="left")

    # shared interests column
    final["shared_interests"] = final["kw_set"].apply(lambda s: shared_kw_str(u_kw, s))

    # pick columns + order
    cols = ["user_id","cupid_final","slot","age","sex","location","job","religion","shared_interests"]
    for c in cols:
        if c not in final.columns:
            final[c] = np.nan
    return final[cols].sort_values(["slot","cupid_final"], ascending=[True, False]).reset_index(drop=True)

# run
final_table = cupid_finalize_table_simple(diverse_slate, user_id=u, k=10, serendipity_n=2)
display(final_table)


,user_id,cupid_final,slot,age,sex,location,job,religion,shared_interests
0,15046,0.890954,matched,36,m,"san francisco, california",artistic / musical / writer,judaism but not too serious about it,"book, food, music"
1,18838,0.854691,matched,29,m,"san francisco, california",computer / hardware / software,catholicism but not too serious about it,"books, movies"
2,3717,0.833243,matched,37,m,"palo alto, california",science / tech / engineering,NaN,"books, music"
3,33504,0.818558,matched,32,m,"san francisco, california",artistic / musical / writer,agnosticism,
4,8486,0.813659,matched,24,m,"walnut creek, california",student,NaN,"books, food, movies, music"
5,38159,0.809769,matched,32,m,"san carlos, california",artistic / musical / writer,NaN,cooking
6,31015,0.807573,matched,31,m,"san francisco, california",other,NaN,"books, cooking, food, music"
7,16307,0.805876,matched,32,m,"san francisco, california",science / tech / engineering,atheism but not too serious about it,"book, books, cooking, movies ..."
8,19933,0.776184,suggested,38,m,"oakland, california",education / academia,buddhism,"books, music"
9,2764,0.774847,suggested,43,m,"san francisco, california",executive / management,christianity and laughing about it,"book, cooking, food, music"


In [29]:
from openai import OpenAI
import os
import requests

In [33]:
# ==== CUPID: pickup lines + question + date idea (API only) ====
API_KEY = "sk-proj-"
os.environ["OPENAI_API_KEY"] = API_KEY

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def truncate_bio(bio: str, max_words: int = 30) -> str:
    """Return the first max_words words of bio (with ellipsis if longer)."""
    words = str(bio).split()
    return " ".join(words[:max_words]) + (" ..." if len(words) > max_words else "")

def cupid_print_starters(user_id: int, k: int = 3, model: str = "gpt-4o-mini"):
    """
    For the top-k CUPID matches:
      - Show CUPID score + profile details
      - Generate 1 pickup line + 1 playful question + 1 date idea
    Assumes `final_slate` already exists from the CUPID pipeline.
    """
    # grab top-k matches from your CUPID final slate
    ranked = final_slate.sort_values("cupid_final", ascending=False).head(k).copy()

    for _, r in ranked.iterrows():
        uid   = int(r["user_id"])
        score = float(r["cupid_final"])
        age   = r.get("age", "")
        sex   = r.get("sex", "")
        loc   = r.get("location", "")
        job   = r.get("job", "")
        rel   = r.get("religion", "")
        # safe bio fetch + truncate
        bio_series = df.loc[df["user_id"] == uid, "bio_text"]
        bio_full = bio_series.iloc[0] if len(bio_series) else ""
        bio = truncate_bio(bio_full, max_words=30)

        # API call to GPT
        prompt = (
            f"Profile bio:\n{bio}\n\n"
            "Write exactly three lines:\n"
            "PICKUP: one playful pickup line (no emojis)\n"
            "QUESTION: one light, open question (no emojis)\n"
            "DATE: one fun first-date idea tailored to their mutual interests\n"
        )
        resp = client.chat.completions.create(
            model=model,
            messages=[{"role":"user","content": prompt}],
            temperature=0.8,
        )
        text = resp.choices[0].message.content.strip().splitlines()

        # extract three parts
        pickup = next((ln.split(":",1)[1].strip() for ln in text if ln.upper().startswith("PICKUP:")), "")
        question = next((ln.split(":",1)[1].strip() for ln in text if ln.upper().startswith("QUESTION:")), "")
        date_idea = next((ln.split(":",1)[1].strip() for ln in text if ln.upper().startswith("DATE:")), "")

        # formatted printout
        print(f"\nCUPID Score = {score:.4f}")
        print(f"User id = {uid} | age = {age} | sex = {sex} | location = {loc}")
        print(f"job = {job} | religion = {rel}")
        print("bio text =")
        print(bio if bio else "(no bio)")
        print("\nGenerated starters:")
        print(f"  Pickup line: {pickup if pickup else '(n/a)'}")
        print(f"  Question:    {question if question else '(n/a)'}")
        print(f"  Date idea:   {date_idea if date_idea else '(n/a)'}")

# run (top 3 only to limit API usage)
cupid_print_starters(user_id=u, k=3, model="gpt-4o-mini")



CUPID Score = 0.8910
User id = 15046 | age = 36 | sex = m | location = san francisco, california
job =  | religion = 
bio text =
grew up near philadelphia. practiced law for a few years before deciding that i wanted to do something more creative. moving to the west coast has exposed me to a ...

Generated starters:
  Pickup line: Are you a legal document? Because you’ve got all the right clauses to catch my interest.
  Question:    What’s one creative project you’ve always wanted to dive into but haven’t yet?
  Date idea:   Let’s explore a local art gallery followed by a DIY painting class to unleash our inner creatives!

CUPID Score = 0.8547
User id = 18838 | age = 29 | sex = m | location = san francisco, california
job =  | religion = 
bio text =
i love to have fun -- this usually happens while having great discussions, watching sports, playing video games, exercising, reading books, watching movies, and many other things i find fascinating. ...

Generated starters:
  Pickup line: A